# Intermediate 2: SQL Aggregations + Machine Learning (Scikit-Learn)

In this notebook, we will:

- Load the same dataset (customers + orders + products)
- Create SQL-style aggregated features using pandas
- Prepare features for ML
- Train a simple classifier to predict high-spending customers


In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

print("Libraries loaded!")


Libraries loaded!


In [4]:
# Customers table
customers = pd.DataFrame({
    "customer_id": [1, 2, 3, 4],
    "customer_name": ["Alice", "Bob", "Charlie", "David"],
    "country": ["USA", "USA", "Canada", "India"]
})

# Orders table
orders = pd.DataFrame({
    "order_id": [101, 102, 103, 104, 105, 106],
    "customer_id": [1, 1, 2, 4, 4, 2],
    "product_id": [1001, 1002, 1001, 1003, 1002, 1003],
    "quantity": [1, 2, 3, 1, 4, 2]
})

# Products table
products = pd.DataFrame({
    "product_id": [1001, 1002, 1003],
    "product_name": ["Laptop", "Mouse", "Keyboard"],
    "price": [1200.0, 25.0, 45.0]
})

print("Dataset loaded!")


Dataset loaded!


In [5]:
# JOIN customers + orders + products
data = pd.merge(customers, orders, on="customer_id", how="inner")
data = pd.merge(data, products, on="product_id", how="inner")

# Add total_amount
data["total_amount"] = data["quantity"] * data["price"]

data


,customer_id,customer_name,country,order_id,product_id,quantity,product_name,price,total_amount
0,1,Alice,USA,101,1001,1,Laptop,1200.0,1200.0
1,1,Alice,USA,102,1002,2,Mouse,25.0,50.0
2,2,Bob,USA,103,1001,3,Laptop,1200.0,3600.0
3,2,Bob,USA,106,1003,2,Keyboard,45.0,90.0
4,4,David,India,104,1003,1,Keyboard,45.0,45.0
5,4,David,India,105,1002,4,Mouse,25.0,100.0


In [6]:
# JOIN customers + orders + products
data = pd.merge(customers, orders, on="customer_id", how="inner")
data = pd.merge(data, products, on="product_id", how="inner")

# Add total_amount
data["total_amount"] = data["quantity"] * data["price"]

data


,customer_id,customer_name,country,order_id,product_id,quantity,product_name,price,total_amount
0,1,Alice,USA,101,1001,1,Laptop,1200.0,1200.0
1,1,Alice,USA,102,1002,2,Mouse,25.0,50.0
2,2,Bob,USA,103,1001,3,Laptop,1200.0,3600.0
3,2,Bob,USA,106,1003,2,Keyboard,45.0,90.0
4,4,David,India,104,1003,1,Keyboard,45.0,45.0
5,4,David,India,105,1002,4,Mouse,25.0,100.0


In [7]:
features = (
    data.groupby(["customer_id", "customer_name", "country"], as_index=False)
        .agg(
            total_spent=("total_amount", "sum"),
            total_orders=("order_id", "nunique"),
            avg_order_value=("total_amount", "mean")
        )
)

features


,customer_id,customer_name,country,total_spent,total_orders,avg_order_value
0,1,Alice,USA,1250.0,2,625.0
1,2,Bob,USA,3690.0,2,1845.0
2,4,David,India,145.0,2,72.5


In [8]:
# Label: 1 = high spender, 0 = low spender
threshold = features["total_spent"].median()
features["high_spender"] = (features["total_spent"] > threshold).astype(int)

features


,customer_id,customer_name,country,total_spent,total_orders,avg_order_value,high_spender
0,1,Alice,USA,1250.0,2,625.0,0
1,2,Bob,USA,3690.0,2,1845.0,1
2,4,David,India,145.0,2,72.5,0


In [9]:
encoder = OneHotEncoder(sparse_output=False)
country_encoded = encoder.fit_transform(features[["country"]])
country_df = pd.DataFrame(country_encoded, columns=encoder.get_feature_names_out(["country"]))

final_df = pd.concat([features.drop(columns=["country", "customer_name"]), country_df], axis=1)
final_df


,customer_id,total_spent,total_orders,avg_order_value,high_spender,country_India,country_USA
0,1,1250.0,2,625.0,0,0.0,1.0
1,2,3690.0,2,1845.0,1,0.0,1.0
2,4,145.0,2,72.5,0,1.0,0.0


In [10]:
X = final_df.drop(columns=["high_spender"])
y = final_df["high_spender"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

preds = model.predict(X_test)
print(classification_report(y_test, preds))


/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1706: Unde

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

